#### 주가 예측 실습

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

saveDir = './Saver/predictStock(reg).save'   # 학습 결과를 저정할 폴더
saveFile = saveDir + '/save'   # 학습 결과를 저장할 파일

In [2]:
sm = pd.read_csv('./stockData/005930.csv', index_col=0, parse_dates=True)
sm = sm.sort_index() # 날짜 순으로 정렬한다
sm = sm.dropna()#결측 행 제거

In [3]:
sm['sma120'] = sm['close'].rolling(window=20).mean()      # 20일 이동평균
sm['sma240'] = sm['close'].rolling(window=60).mean()      # 40일 이동평균
sm = sm.drop('volume', 1)                                 # volume 컬럼을 제거한다
sm = sm / 10000

In [33]:
sm.tail()

,open,high,low,close,sma120,sma240
date,,,,,,
2019-06-19,4.545,4.545,4.500,4.535,4.36525,4.455667
2019-06-20,4.485,4.550,4.485,4.550,4.37525,4.455667
2019-06-21,4.575,4.580,4.520,4.570,4.38450,4.456417
2019-06-24,4.520,4.580,4.520,4.550,4.39850,4.456667
2019-06-25,4.520,4.580,4.520,4.545,4.41250,4.457667


In [34]:
sm['next close'] = sm['close'].shift(-1)
sm = sm.dropna()

In [35]:
sm['class'] = 0.0
sm.loc[sm['next close'] > sm['close'], 'class'] = 1.0
sm.head()

,open,high,low,close,sma120,sma240,next close,class
date,,,,,,,,
2011-08-04,1.678,1.678,1.630,1.642,1.6883,1.718200,1.578,0.0
2011-08-05,1.584,1.610,1.578,1.578,1.6787,1.715000,1.520,0.0
2011-08-08,1.540,1.578,1.520,1.520,1.6682,1.709800,1.448,0.0
2011-08-09,1.500,1.506,1.392,1.448,1.6556,1.703900,1.440,0.0
2011-08-10,1.520,1.520,1.440,1.440,1.6433,1.698233,1.416,0.0


In [36]:
sm = sm.sample(frac=1)
sm.head()

,open,high,low,close,sma120,sma240,next close,class
date,,,,,,,,
2014-06-13,2.804,2.808,2.726,2.734,2.86260,2.755000,2.748,1.0
2019-01-24,4.305,4.310,4.235,4.305,4.01325,4.140417,4.475,1.0
2012-10-26,2.612,2.624,2.574,2.574,2.66020,2.592733,2.598,1.0
2012-05-10,2.658,2.672,2.644,2.654,2.64140,2.521867,2.606,0.0
2014-02-20,2.580,2.592,2.562,2.572,2.57310,2.706433,2.660,1.0


In [37]:
len(sm.loc[sm['class'] == 0])

962

In [39]:
inputX = np.array(sm.iloc[:1500, 0:6])
outputY = np.array(sm.iloc[:1500, 7], dtype=np.float32).reshape(-1,1)

testX = np.array(sm.iloc[1500:, 0:6])
testY = np.array(sm.iloc[1500:, 7], dtype=np.float32).reshape(-1,1)

In [44]:
nInput = 6      # input layer의 neuron 개수
nHidden1 = 20   # hidden layer-1의 neuron 개수
nOutput = 1     # output layer의 neuron 개수

In [48]:
# 그래프를 생성한다
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[None, nInput], name='x')
y = tf.placeholder(tf.float32, shape=[None, nOutput], name='y')

# hidden layer의 Weight (Wh)와 Bias (Bh)
Wh1 = tf.Variable(tf.truncated_normal([nInput, nHidden1]), dtype=tf.float32, name='Wh1')
Bh1 = tf.Variable(tf.zeros(nHidden1), dtype=tf.float32, name='Bh1')

# output layer의 Weight (Wo)와 Bias (Bo)
Wo = tf.Variable(tf.truncated_normal([nHidden1, nOutput]), dtype=tf.float32, name='Wo')
Bo = tf.Variable(tf.zeros(nOutput), dtype=tf.float32, name='Bo')

# Hidden layer의 출력값. activation function은 relu
H1 = tf.nn.relu(tf.add(tf.matmul(x, Wh1), Bh1), name='H1')

# Output layer의 출력값. activation function은 sigmoid
predY = tf.nn.sigmoid(tf.add(tf.matmul(H1, Wo), Bo), name='predY')

# Cost function 정의. MSE
# cost = tf.reduce_mean(tf.square(tf.square(tf.subtract(predY, y))))

# Cost function CE
clipY = tf.clip_by_value(predY, 0.000001, 0.99999)  # log(0)를 방지한다
cost = -tf.reduce_mean(y * tf.log(clipY) + (1-y) * tf.log(1-clipY))

# 학습
alpha = tf.placeholder(tf.float32, name='alpha')
optimizer = tf.train.AdamOptimizer(alpha)
train = optimizer.minimize(cost)

saver = tf.train.Saver()

In [49]:
# 그래프를 실행한다
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 기존 학습 결과를 적용한 후 추가로 학습한다
#if tf.train.checkpoint_exists(saveDir):
#    saver.restore(sess, saveFile)

In [ ]:
# Mini-batch 적용
nBatchCnt = 5
nBatchSize = int(inputX.shape[0] / nBatchCnt)
for i in range(10000):
    # Mini-batch 방식으로 학습한다
    for n in range(nBatchCnt):
        # input 데이터를 Mini-batch 크기에 맞게 자른다
        nFrom = n * nBatchSize
        nTo = n * nBatchSize + nBatchSize

        # 마지막 루프이면 nTo는 input 데이터의 끝까지.
        if n == nBatchCnt - 1:
            nTo = inputX.shape[0]
            
        bx = inputX[nFrom : nTo]
        by = outputY[nFrom : nTo]
        
        _, xcost = sess.run([train, cost], feed_dict={x: bx, y: by, alpha: 0.001})
    
    if i % 1000 == 0:
        print('cost = %.6f' % xcost)
        
# 학습 결과를 저장해 둔다
#saver.save(sess, saveFile)

cost = 0.698436
cost = 0.694307
cost = 0.692853
cost = 0.692129


In [15]:
# 학습이 완료되면, Wh, Bh, Wo, Bo 이 모두 업데이트 되었으면, inputX를 넣어서 출력값을 확인한다.
# 출력값은 inputY의 추정값이다.
yHat = sess.run(predY, feed_dict={x: testX})

In [16]:
yHat[np.where(yHat > 0.5)] = 1
yHat[np.where(yHat < 0.5)] = 0

In [17]:
yHat = np.array([1 if  x > 0.5 else 0 for x in yHat]).reshape(-1,1)

In [18]:
accuracy = (testY == yHat).sum() / len(testX)

In [19]:
print("Accuracy = %.4f" % accuracy)

Accuracy = 0.4890
